### Comparación e Identificación de Voz

Identificar un hablante comparando su voz contra las voces registradas en la base de datos.

**Pasos:** Configurar rutas → Cargar modelo → Grabar audio → Generar embedding → Cargar BD → Calcular similitud → Identificar (threshold 0.75)

#### 1. Validar modelo y configurar rutas
Verifica que el modelo existe y define directorio base.

In [1]:
import os

BASE_DIR = os.path.abspath("..")

MODEL_DIR = os.path.join(
    BASE_DIR,
    "models",
    "spkrec-ecapa-voxceleb"
)

assert os.path.exists(MODEL_DIR), "Modelo no encontrado"
assert os.path.exists(os.path.join(MODEL_DIR, "hyperparams.yaml")), "hyperparams.yaml faltante"

print("Modelo listo en:", MODEL_DIR)


Modelo listo en: d:\work_jhonatan_becerra\speach-recognition\models\spkrec-ecapa-voxceleb


#### 2. Cargar modelo ECAPA-TDNN
Carga el modelo SpeechBrain con directorio cache temporal.

In [2]:
from speechbrain.pretrained import EncoderClassifier

classifier = EncoderClassifier.from_hparams(
    source=MODEL_DIR,
    savedir=os.path.join(MODEL_DIR, "_cache"),
    run_opts={"device": "cpu"}
)

print("Modelo cargado correctamente")


d:\work_jhonatan_becerra\speach-recognition\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
d:\work_jhonatan_becerra\speach-recognition\venv\Lib\site-packages\speechbrain\utils\torch_audio_backend.py:22: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
torchvision is not available - cannot save figures
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
d:\work_jhonatan_becerra\speach-recognition\venv\Lib\site-packages\speechbrain\utils\torch_audio_backend.py:22: UserWarning: torchaudio._backend.set_audio_backend has b

Modelo cargado correctamente


#### 3. Configurar parámetros de captura
16 kHz, mono, 10 segundos.

In [ ]:
import os
import time
import torch
import torchaudio
import numpy as np

SAMPLE_RATE = 16_000
CHANNELS = 1
DURATION_SEC = 10

AUDIO_DIR = os.path.join(BASE_DIR, "audio")
os.makedirs(AUDIO_DIR, exist_ok=True)

AUDIO_PATH = os.path.join(AUDIO_DIR, "audio_10s.wav")


#### 4. Grabar audio del hablante a identificar
Captura 10 segundos de audio desde el micrófono.

In [12]:
import sounddevice as sd
import soundfile as sf

print("Grabando audio por 10 segundos...")
audio = sd.rec(
    int(DURATION_SEC * SAMPLE_RATE),
    samplerate=SAMPLE_RATE,
    channels=CHANNELS,
    dtype="float32"
)
sd.wait()

sf.write(AUDIO_PATH, audio, SAMPLE_RATE)
print("Audio guardado en:", AUDIO_PATH)


Grabando audio por 10 segundos...
Audio guardado en: d:\work_jhonatan_becerra\speach-recognition\audio\audio_10s.wav


#### 5. Procesar y normalizar waveform
Valida sample rate y duración, convierte a mono y normaliza.

In [13]:
waveform, sr = torchaudio.load(AUDIO_PATH)

assert sr == SAMPLE_RATE, f"Sample rate inválido: {sr}"
assert waveform.shape[1] >= SAMPLE_RATE * DURATION_SEC * 0.95, "Audio demasiado corto"

waveform = waveform.mean(dim=0, keepdim=True)  # mono
waveform = waveform / torch.max(torch.abs(waveform))  # normalización

print("Waveform shape:", waveform.shape)


Waveform shape: torch.Size([1, 160000])


#### 6. Generar embedding del audio capturado
Extrae vector de 192 dimensiones del audio a identificar.

In [14]:
with torch.no_grad():
    embedding = classifier.encode_batch(waveform)

embedding_actual = embedding.squeeze().cpu().numpy()

print("Embedding generado. Shape:", embedding_actual.shape)


Embedding generado. Shape: (192,)


#### 7. Cargar base de datos de voces
Lee todos los archivos .npy de voices_db (cada archivo = una persona registrada).

In [15]:
VOICES_DB = os.path.join(BASE_DIR, "voices_db")
os.makedirs(VOICES_DB, exist_ok=True)

voice_pool = {}

for file in os.listdir(VOICES_DB):
    if file.endswith(".npy"):
        name = file.replace(".npy", "")
        voice_pool[name] = np.load(os.path.join(VOICES_DB, file))

print("Voces cargadas:", list(voice_pool.keys()))


Voces cargadas: ['freddy', 'jhonatan']


#### 8. Calcular similitud coseno
Compara embedding actual vs todos los registrados (valores cercanos a 1 = misma persona).

In [16]:
from numpy.linalg import norm

def cosine_similarity(a, b):
    return np.dot(a, b) / (norm(a) * norm(b))

scores = {}

for name, emb in voice_pool.items():
    scores[name] = cosine_similarity(embedding_actual, emb)

scores


{'freddy': 0.16063125, 'jhonatan': 0.8501665}

#### 9. Identificar hablante con threshold
Score >= 0.75 = identifica persona | Score < 0.75 = desconocido.

In [17]:
THRESHOLD = 0.75

if scores:
    best_match = max(scores, key=scores.get)
    best_score = scores[best_match]

    if best_score >= THRESHOLD:
        result = f"IDENTIDAD: {best_match} (score={best_score:.3f})"
    else:
        result = f"DESCONOCIDO (mejor score={best_score:.3f})"
else:
    result = "BD VACÍA – no hay voces registradas"

result


'IDENTIDAD: jhonatan (score=0.850)'